# Where is a hospital in Manhattan Downtown?

In this lab we will create 2 vector indices to answer a very simple question: if you are in Manhattan downtown, where is the nearest hospital? We will base our soultion on two sources of data:
- [Points of Interest dataset](https://drive.google.com/file/d/1LUudtCADqSxRl18ZzCzyPPGfhuUo2ZZs/view?usp=sharing). This is a 10% sample of a bigger dataset. Download and uncompress the file.
- [Google Geocoding API](https://developers.google.com/maps/documentation/geocoding/start) or any other [equivalent service](https://gisgeography.com/geocoders/). For Google you will need to obtain a key. **PLEASE DO NOT SUBMIT THE KEY TO MOODLE :)**


# 0. Prepare data

In [ ]:
import os


def draw_earth(xlim=(-180, +180), ylim=(-90, +90)):
    import matplotlib.pyplot
    import pandas as pd
    import json
    plt.figure(figsize=(15, 8))
    plt.xlim(xlim)
    plt.ylim(ylim)
    
    # this file also lives in github. Adjust the path if needed.
    df = pd.read_csv("world.csv")
    
    for row in df['geojson']:
        js = json.loads(row)
        polys = js['coordinates']
        for poly in polys:
            for pp in poly:
                x, y = [v[0] for v in pp], [v[1] for v in pp]
                plt.plot(x, y, color='gray')

Reading the dataset and storing coordinates in `GEO` matrix:

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

# replace filename if you want to use another data file
# be careful! 2M points is still a big number and can eat significant amout of memory
with open("poi_sample01.pickle", "rb") as f:
    dataset = pickle.load(f)

# let's sample 20000 of points to draw
step = len(dataset) // 20000

# pure coordinated in compressed representation, 2B per number -> 8MB per array
#GEO = np.array([v[0] for v in dataset], dtype=np.float16)
GEO = np.array([v[0] for v in dataset], dtype=np.float16) + np.random.normal(0, 0.01, np.array([v[0] for v in dataset], dtype=np.float16).shape)
N = len(dataset)
# free the memory
dataset = None
import gc
gc.collect()

Showing approximate dataset data distribution:

In [ ]:
draw_earth(ylim=(-75, 75))
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.grid()
plt.scatter(GEO[::step, 0], GEO[::step, 1])
plt.show()

## 0.1. [1 point] Ok. Let's prepare ourselves to read the data from the hard drive

We will prepare id-based shards (data will be distributed into equal files with ranges `[0..capacity-1], [capacity..2*capacity-1], ...`. Each shard will store `capacity` elements. Your task is to complete the implementation with `iterate_dataset` function.

In [ ]:
def split_shards(file, folder='shard', capacity=20000):
    import pickle, os, math, gc
    if not os.path.exists(folder):
        os.mkdir(folder)
    with open(file, "rb") as f:
        dataset = pickle.load(f)
    nshards = len(dataset) // capacity
    if nshards * capacity < len(dataset):
        nshards += 1
    
    for i in range(nshards):
        with open(f"{folder}/{i}", 'wb') as f:
            part = dataset[i * capacity:(i+1)*capacity]
            pickle.dump(part, f)
    dataset = None
    gc.collect()            

    
def dataset_get(indices, folder='shard', capacity=20000) -> list:
    result = []
    groups = {}
    for i in indices:
        x = i // capacity
        if x not in groups:
            groups[x] = []
        groups[x].append(i)
    for x in groups:
        with open(f"{folder}/{x}", "rb") as f:
            sha = pickle.load(f)
            for i in groups[x]:
                row = sha[i % capacity]
                result.append(row)
    return result


# should return iterator, which goes through all elements, consequently opening files
# use ``yield`` operator to simplify your code
def iterate_dataset(items, folder="shard", capacity=20000):
    nshards = items // capacity
    if nshards * capacity < items:
        nshards += 1
    for n in range(nshards):
        with open(f"{folder}/{n}", "rb") as f:
            sha = pickle.load(f)
            for elem in sha:
                yield (elem, "stub")

In [ ]:
split_shards("poi_sample01.pickle")

asserts:

In [ ]:
i = 0
for r in iterate_dataset(N):
    i += 1

assert i == N

asserts:

In [ ]:
import numpy as np

for i in [137, 40000, 600000]:
    assert np.allclose(GEO[i,:], dataset_get([i])[0][0], atol=5*1e-2), ""

In [ ]:
dataset_get([1, 10, 1000234, N-1])

---

# And now the task!
Ok. We are ready to perform the following steps utilizing functions beyond:
1. Build **coordinate search index**. We will use it to obtain POI from the given region.
3. Implement **vector text embedding index** (Annoy, HNSW) to serve semantic queries.
3. Implement **geocoding** with cache. We will use it to obtain city coordinates.
4. Impement search for **double queries: town and location type**.

# 1. Create spacial index for points of interest

We will store dataset rows numbers as values, and coordinates as keys. Please use [KDtree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree) or [BallTree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html#sklearn.neighbors.BallTree) from sklearn.

## 1.1. [5 points] Build the index and return it

Implement the following functions. To get the full grade:
- `build_geospacial_index` should build and return a search tree object: KDTree or BallTree.
- `kNN` accepts a 2D-point, `k` neighbours parameter, and returns **approximate** `k` neighbours (they can be different from the real neighbours).
- `inRadius` accepts a 2D-point, L<sub>2</sub> `radius`, and returns points inside the radius. Clarification: for simplicity **radius is given in units of coordinates (degrees)**, not kilometers or meters.

In [ ]:
from sklearn.neighbors import BallTree

def build_geospacial_index(points, leaf_size=5) -> BallTree: # or KDTree here
    tree = BallTree(points, leaf_size)
    return tree


def kNN(query_point: list, k: int, index: BallTree) -> list:
    _, ind = index.query([query_point], k=k)
    return ind


def inRadius(query_point: list, r: float, index: BallTree) -> list:
    ind = index.query_radius([query_point], r=r)
    return ind[0]

In [ ]:
import numpy as np
import time
start = time.time()
for i in range(len(GEO)):
    coorx, coory = np.random.normal(0.0, 0.01,2)
    if i%10000 == 0:
        print(GEO[i], [GEO[i][0] + coorx, GEO[i][1] + coory])
    GEO[i] = [GEO[i][0] + coorx, GEO[i][1] + coory]

print(time.time() - start)
#print(random.uniform(0.0, 0.01))
print(np.random.normal(0.0, 0.01,2))
print(np.random.normal(0.0, 0.01))
print(np.random.normal(0.0, 0.01))
print(np.random.normal(0.0, 0.01))
#GEO = np.array([v[0] for v in dataset], dtype=np.float16) + np.random.normal(0, 0.01, np.array([v[0] for v in dataset], dtype=np.float16).shape)

In [ ]:
spaidx = build_geospacial_index(GEO)

In [ ]:
test_id = 13

idx = kNN(GEO[test_id], 10, spaidx)
print(sorted(idx))
assert test_id in idx, "Point itself should be in results"

idx = inRadius(GEO[test_id], 0.0625, spaidx)
print(sorted(idx))
assert test_id in idx, "Point itself should be in results"

## 1.2. [4 points] Tricky assert

Some keys (coordinates) in the dataset (surprise!) are duplicates. Unfortunately search trees (in basic implemenation) cannot support duplicates. Thus you can follow one of the strategies:
- a key (coordinateS) corresponds to multiple values. This may require additional data strictures.
- improve the data (coordinates) to avoid collisions (e.g. make sure they never coinside by adding insignificant noise)

Pass the assert to get full points.

In [ ]:
points = [1966663, 1480877, 2126566]
for p in points:
    x = GEO[p, :]
    r = kNN(x, 1000, spaidx)
    assert (p in r), "Query did not return itself"

## 1.3. [Optional demo] How leaf size influences build and search speed?

Let us look at how parameter of leaf size affects speed of search and construction

In [ ]:
import random
import time
import matplotlib.pyplot as plt
import tqdm

queries = random.sample(range(N), 1000)
leaf_sizes = [1, 10, 20, 50]

build_times = []
query_times = []
for ls in tqdm.tqdm(leaf_sizes):
    start = time.time()
    idx = build_geospacial_index(GEO, ls)
    build_times.append(time.time() - start)
    
    start = time.time()
    for q in queries:
        d, r = spaidx.query([GEO[q]], 10000, sort_results=False, breadth_first=True)
    query_times.append(time.time() - start)
    idx = None
    gc.collect()

plt.xlabel("build time, s")
plt.ylabel("query time, s")
plt.scatter(build_times, query_times)
for i, ls in enumerate(leaf_sizes):
    plt.annotate(str(ls), (build_times[i], query_times[i]))

## 1.4. [5 points] Range queries?

Ok, you have a **radius query**, but what about **rectangual ranges**? Using the functions you already wrote, please, implement the range query given `north-east` and `south-west` corners. Pass the asserts to get points.

In [ ]:
import numpy as np
from scipy.spatial import distance

# should return ids of the rows in this range
def get_in_range(ne, sw, spacial_index, GEO) -> list:
    r = np.sqrt((ne[0]-sw[0])**2 + (sw[1]-ne[1])**2)/2
    centerx = sw[0] + (np.abs(ne[0]- sw[0])) / 2
    centery =  sw[1] + (np.abs(sw[1]-ne[1])) / 2
    idx = inRadius([centerx,centery], r, spacial_index)
    rect_range = []
    for elem in idx:
        if sw[0]<= GEO[elem][0] <= ne[0] and sw[1] <= GEO[elem][1] <= ne[1]:
            rect_range.append(elem)

    return rect_range

In [ ]:
ids = get_in_range([-73.97, 40.75], [-74.03, 40.70], spaidx, GEO)

In [ ]:
def print_starbucks(ids):
    for row in dataset_get(ids):
        if 'Starbucks' in row[1]:
            print(row[1])

asserts:

In [ ]:
ids = get_in_range([-73.97, 40.75], [-74.03, 40.70], spaidx, GEO)
print(ids)
assert any(map(
            lambda x: 'Manhattan, 80 Delancey St' in x[1], 
            dataset_get(ids))), "This Starbucks should be in place!"

print_starbucks(ids)

# 2. Geocoding

In this block we will learn, how to convert text place names into coordinate rectangles.

## 2.1. [5 points] Implement geocoding
which returns north-eastern and south-western points of the place. Pass asserts to get full points.

In [ ]:
import requests
import json
# this function returns a pair of tuples: NE and SW corners.
def get_town_range_coordinates(town: str, google_maps_api_key: str) -> tuple:
    api = f"http://api.positionstack.com/v1/forward?access_key={google_maps_api_key}&query={town}&bbox_module=1"
    text = requests.get(api).text
    data = json.loads(text)["data"][0]
    location = data["bbox_module"]
    return [location[2], location[3]], [location[0], location[1]]

If needed, request your key here: https://developers.google.com/maps/documentation/geocoding/get-api-key

Note, that from the **local machine** you may (!) need to have VPN enabled. For **colab** this should work smoothly.

In [ ]:
my_google_maps_api_key = open('google.key', 'r').read()

asserts:

In [ ]:
p = get_town_range_coordinates('Pittsburgh downtown', my_google_maps_api_key)
#print(p)
assert p[1][0] <= -80. <= p[0][0] and p[1][1] <= 40.44 <= p[0][1] 

## 2.2. [5 points] Town queries

Now, having a range query and geocoding, we can implement town-queries! Pass the assert to get the points.

In [ ]:
# should return dataset indices
def get_in_town(town, index, GEO, maps_api_key) -> list:
    ne, sw = get_town_range_coordinates(town, maps_api_key)
    idx = get_in_range(ne,sw, index, GEO)
    return idx

In [ ]:
ids = get_in_town('Pittsburgh downtown', spaidx, GEO, my_google_maps_api_key)

assert any(map(
            lambda x: 'US, Pittsburgh, 810 River Ave' in x[1], 
            dataset_get(ids))), "This Starbucks should be in place!"

print_starbucks(ids)

## 2.3. [5 points] Caching

Why should you pay for every geocaching request, if you can cache them? Implement a cached version on geocoding. The second query does not use internet.

In [ ]:
GEO_CACHE = {}

In [ ]:
global GEO_CACHE

def get_town_range_coordinates_cached(town: str, maps_key: str) -> tuple:
    global GEO_CACHE
    if town in GEO_CACHE.keys():
        print("Cache Hit!")
        return GEO_CACHE[town]
    idx = get_town_range_coordinates(town, maps_key)
    GEO_CACHE[town] = idx
    return idx


def get_in_town_cached(town: str, index, GEO, maps_key: str) -> list:
    if town in GEO_CACHE.keys():
        print("Cache Hit!")
        return GEO_CACHE[town]
    idx = get_in_town(town, index, GEO, maps_key)
    GEO_CACHE[town] = idx
    return idx

In [ ]:
ids = get_in_town_cached('Boulder, CO', spaidx, GEO, my_google_maps_api_key)
print_starbucks(ids)
ids = get_in_town_cached('Boulder, CO', spaidx, GEO, my_google_maps_api_key)
print_starbucks(ids)

# 3. Text search

We are done with geography, but we have no clear method to search for categories. What if we prepare vector index of location names?

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')
names = []

## 3.1. [5 points] Embedding

Here is the trick. If you use any embedding model "as it is", it may take some hours to prepare 2M embeddings. It's ok if you can wait (and get **2 points**), but...

To get full points, please think, how you can speed up the process with embedding to less than 5 minutes?

HINT: spacy model `nlp` has a [dictionary for word embeddings](https://spacy.io/api/vocab). You can access `nlp.vocab[word].vector` to get word embedding, `nlp.vocab.strings` map stores integer indices. Or maybe you want to use `fasttext` instead?

In [70]:
import numpy as np

# if you need most frequent english words for some reason.
WORDS = set([a.strip() for a in open('words.txt', 'r').readlines()])

def embed(text, nlp):
    # TODO your code here. This should return a vector (300,)
    arr = []
    words = text.split()
    for w in words:
        nlp.vocab[w]
        if w in nlp.vocab:
            arr.append(nlp.vocab[w].vector)
    sem = np.mean(arr, axis=0)
    #print(sem)
    return sem / np.linalg.norm(sem)

In [ ]:
from tqdm import tqdm
import gc
import numpy as np

embeddings = np.zeros((N, 300), dtype=np.float16)

for i, item in enumerate(tqdm(iterate_dataset(N), total=N)):
    name = item[1].split('.')[0]
    emb = embed(name, nlp)
    if emb is not None:
        embeddings[i, :] = emb
gc.collect()

## 3.2. [10 points] Vector index

Here you build vector index for our embeddings. I want to warn Windows users, that they can observe problems with installing Faiss and HNSWlib (please refer to the corresponding lab). Still this is not the reason not to try :)
Choose **one of the libraries** and fulfill the requirements to get full points:
1. If you choose [FAISS](https://faiss.ai/). Get started with [installation](https://faiss.ai/#install) and this [tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started). To get full points your index must use [Product Quantization](https://github.com/facebookresearch/faiss/wiki/Lower-memory-footprint): 50 subvectors, 8 bits (1 byte) each. Use custom `nprobe` parameter equal to 23. Number or Voronoi cells is `65536`. Refer to [this document](https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index#if-1m---10m-ivf65536_hnsw32) to understand recommendations.
2. If you use [HNSWlib](https://github.com/nmslib/hnswlib) (or [nmslib](https://github.com/nmslib/nmslib)) then follow these requirements. Use `cosine` metric for index construction, maximum number of outgoing connections (max outdegree) in the graph is 16, `ef` parameter at construction time should be `250`. Some useful information is given [here](https://github.com/nmslib/nmslib/blob/master/manual/methods.md).
3. For [Annoy](https://github.com/spotify/annoy) you should use cosine distance for the space (if vectors are normed, you can use dot product intead), use all CPU cores at construction time. Build the index right on the disk, then load. Your index should consist of 37 trees.

**NB** If you run on not-very-modern hardware (e.g. your RAM is less then 8GB), then you'd better reduce dataset size (e.g. take a specific region only like US east cost). You can also reduce other parameters only for the sake of RAM efficiency, but please specify and justify your decisions.

e.g.
```    
roi = set(get_in_range([-68.645945, 43.163175], [-80.461502, 37.097044], spaidx, GEO))
```


**HINT** You can remove `embeddings` array and call `gc.collect()` before loading index to RAM.

**HINT2** Should you index vectors with $[0]^N$ values?

In [72]:
embedding_index = None

In [78]:
import numpy as np
import hnswlib
def get_vector_index():
    # TODO returns index object
    ind = hnswlib.Index("cosine", 300)
    ind.init_index(N, 16, 250)
    ind.add_items(embeddings)
    return ind


def get_kNN_embeddings(embedding, k, index):
    # TODO write your kNN queries here
    labels, distances = index.knn_query(embedding, k)
    return distances[0]

In [74]:
start = time.time()
embedding_index = get_vector_index()
print(time.time() - start)

1004.9357216358185


In [71]:
print(embeddings)

[[-0.02176  0.1177   0.00952 ... -0.03397 -0.02159  0.03308]
 [-0.02176  0.1177   0.00952 ... -0.03397 -0.02159  0.03308]
 [-0.02176  0.1177   0.00952 ... -0.03397 -0.02159  0.03308]
 ...
 [-0.02176  0.1177   0.00952 ... -0.03397 -0.02159  0.03308]
 [-0.02176  0.1177   0.00952 ... -0.03397 -0.02159  0.03308]
 [-0.02176  0.1177   0.00952 ... -0.03397 -0.02159  0.03308]]


In [59]:
print(len(embed('pharmacy', nlp)))

300


In [80]:
result = get_kNN_embeddings(embed('pharmacy', nlp), 1000, embedding_index)

assert len(result) == 1000

# 4. [5 points] And now we want to have this together!

Say no more:

In [83]:
def find(town, query) -> list:
    data = get_in_town_cached(town, spaidx, GEO, my_google_maps_api_key)
    return

In [84]:
items = find('Manhattan downtown', 'hospital')
print(items[:20])
xy = np.array([row[0] for row in items])

IndexError: list index out of range

In [ ]:
NE, SW = get_town_range_coordinates_cached('Manhattan downtown', my_google_maps_api_key)
draw_earth(xlim=(SW[0] - 5, NE[0] + 5), ylim=(SW[1] - 5, NE[1] + 5))
plt.scatter(xy[:, 0], xy[:, 1])
plt.show()